In [1]:
BASE_MODEL_DIRECTORY = "meta-llama"
BASE_MODEL_NAME = "Meta-Llama-3-8B-Instruct"
BASE_MODEL_IDENTIFIER = f"{BASE_MODEL_DIRECTORY}/{BASE_MODEL_NAME}"
MODEL_PATH = f"/content/drive/MyDrive/diploma-llm/models/{BASE_MODEL_NAME}/model"
TOKENIZER_PATH = f"/content/drive/MyDrive/diploma-llm/models/{BASE_MODEL_NAME}/tokenizer"
PEFT_MODEL_PATH = f"/content/drive/MyDrive/diploma-llm/models/{BASE_MODEL_NAME}-ft/model"

In [2]:
!pip install torch
!pip install transformers
!pip install peft
!pip install bitsandbytes
!pip install accelerate
!pip install trl
!pip install ngrok

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [3]:
import os
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load the model
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)
model.config.use_cache = False
model.config.pretraining_tp = 1

model = PeftModel.from_pretrained(model, PEFT_MODEL_PATH)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from string import Template
from transformers import pipeline

INPUT_TEMPLATE = Template('тема:$topic\n|ключові слова:$keywords\n|розділ:$chapter:\n')

def generate_text(topic, keywords, chapter, model, tokenizer, max_length, repetiotion_penalty):

  input = INPUT_TEMPLATE.substitute(topic=topic, keywords=keywords, chapter=chapter)

  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_length,
                repetition_penalty=repetiotion_penalty)

  result = pipe(input)

  text = ("".join(part['generated_text'] for part in result)).split(tokenizer.eos_token)[0]
  text = text.replace(input, "")

  return {"text": text}

In [6]:
import ngrok
from google.colab import userdata

ngrok.forward(8000, authtoken=userdata.get('ngrokToken'))

<Task pending name='Task-1' coro=<wrap() running at ngrok_wrapper:6>>

In [ ]:
import json
import threading
from http.server import BaseHTTPRequestHandler, HTTPServer

class SimpleHTTPRequestHandler(BaseHTTPRequestHandler):

    def do_POST(self):
        # Read the length of the content
        content_length = int(self.headers['Content-Length'])
        # Read the content
        post_data = self.rfile.read(content_length)

        # Attempt to parse JSON
        try:
            data = json.loads(post_data)
        except json.JSONDecodeError:
            self.send_response(400)
            self.end_headers()
            self.wfile.write(b"Bad request: Could not decode JSON")
            return

        if self.path == '/generation':
            response = generate_text(data['topic'], data['keywords'], data['chapter'], model, tokenizer, data['max_length'], data['repetition_penalty'])
        else:
            response = {
                'status': 'success',
                'message': 'POST data received, but no handler found',
                'yourData': data
            }

        # Send OK response
        self.send_response(200)
        self.send_header('Content-type', 'application/json')
        self.end_headers()

        self.wfile.write(json.dumps(response).encode('utf-8'))


def run(server_class=HTTPServer, handler_class=SimpleHTTPRequestHandler, port=8000):
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)
    print(f"Starting httpd server on port {port}")
    httpd.serve_forever()

run()
# threading.Thread(target=run).start()


Starting httpd server on port 8000


In [ ]:
!chmod +x ./request.sh

In [ ]:
!./request.sh

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyFo

{"text": "\u0442\u0435\u043c\u0430:\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043d\u0435 \u0437\u0430\u0431\u0435\u0437\u043f\u0435\u0447\u0435\u043d\u043d\u044f \u0434\u043b\u044f \u0434\u043e\u0441\u043b\u0456\u0434\u0436\u0435\u043d\u043d\u044f \u0447\u0430\u0441\u043e\u0432\u0438\u0445 \u0440\u044f\u0434\u0456\u0432\n|\u043a\u043b\u044e\u0447\u043e\u0432\u0456 \u0441\u043b\u043e\u0432\u0430:\u043f\u0440\u043e\u0433\u0440\u0430\u043c\u0430, \u0447\u0430\u0441\u043e\u0432\u0456 \u0440\u044f\u0434\u0438, \u0441\u0442\u0430\u0442\u0438\u0441\u0442\u0438\u043a\u0430, \u043f\u0430\u0440\u0430\u043b\u0435\u043b\u044c\u043d\u0456 \u043e\u0431\u0447\u0438\u0441\u043b\u0435\u043d\u043d\u044f, \u043a\u0435\u0440\u0456\u0432\u043d\u0438\u0446\u0442\u0432\u043e \u043a\u043e\u0440\u0438\u0441\u0442\u0443\u0432\u0430\u0447\u0430, \u043f\u0440\u043e\u0433\u0440\u0430\u043c\u0443\u0432\u0430\u043d\u043d\u044f, \u0434\u043e\u0434\u0430\u0442\u043e\u043a, \u0440\u043e\u0437\u0440\u043e\u0431\u043a\u

In [ ]:
import requests
import json

url = 'http://localhost:8000/generation'
payload = {"topic":"Програмне забезпечення для дослідження часових рядів","keywords":["програма","часові ряди","статистика","паралельні обчислення","керівництво користувача","програмування","додаток","розробка"],"chapter":"Розгортання","max_length":2000,"repetition_penalty":1.2}

response = requests.post(url, json=payload)

if response.status_code == 200:
    data = response.json()
    print(data['text'])
else:
    print("Error:", response.status_code)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyFo

тема:Програмне забезпечення для дослідження часових рядів
|ключові слова:програма, часові ряди, статистика, паралельні обчислення, керівництво користувача, програмування, додаток, розробка
|розділ:Розгортання:
- Структура програми:
	+ Вхідні дані: файл з часовими рядом та описом змінних.
	+ Обробка даних: пошуки періодичності, фільтрація шуму, виявлення тренду.
	+ Відображення результатів: графічний інтерфейс з можливістю відображення різних типів графіку (линійний, логарифмічний, катастрофічно).
	+ Експортування результатів: можливість експортувати отримані результати до формату CSV або Excel.

- Функції:
	1. Пошук періоду зв'язок між змінними.
	2. Фільтрування шуму за допомогою методу медіанного руху.
	3. Визначення тренду за допомогою методу лінійної регресії.
	4. Генерація звітів про результати аналізу.

- Рішення проблеми:
	* Для пошуку періоду зв'язок між змінними буде використана функція `pandas` для створення матриці кореляцій.
	* Для фільтрації шума буде використано алгоритм м

127.0.0.1 - - [19/May/2024 00:54:53] "POST /generation HTTP/1.1" 200 -
